### Import

In [2]:
!pip install category_encoders
!pip install catboost
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.5 MB/s eta 0:00:00


In [3]:
import random
import pandas as pd
import numpy as np
from datetime import datetime
import os
import json
import re

import optuna
import functools

from sklearn.linear_model import (
    LogisticRegression
)

from sklearn.ensemble import (
    ExtraTreesClassifier,
    RandomForestClassifier
)

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score
)
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

import category_encoders as ce

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings(action='ignore')

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


### Data Load

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os

# 현재 작업 디렉터리 출력
print(os.getcwd())

data_path = os.path.join(os.getcwd(), 'drive', 'MyDrive', 'Aimers_6th', 'Data')
print(data_path)

/content
/content/drive/MyDrive/Aimers_6th/Data


In [6]:
config = {
    'root': data_path
    , 'train_path': f'{data_path}/train.csv'
    , 'test_path': f'{data_path}/test.csv'
    , 'submit_path': f'{data_path}/sample_submission.csv'
    , 'seed_list': [42]
    , 'k_fold': 5
    , 'model': 'lgb'       # cbt, logistic, et, rf, lgb, xgb
    , 'encoding': None # None(lgb, xgb, cbt), target, one-hot, ordinal, catboost
    , 'is_ivf': True # 'IVF' if is_ivf else 'DI'

}

### Function

In [12]:
def set_seed(seed: int):
    # Set the seed for reproducibility.
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)


def read_data(config):
    # Load training, testing, and submission CSV files
    df_train = pd.read_csv(config['train_path']).drop(columns=['ID'])  # train data
    df_test = pd.read_csv(config['test_path']).drop(columns=['ID'])    # test data
    df_sub = pd.read_csv(config['submit_path'])

    print(f'train data 수: {df_train.shape[0]}')
    print(f'test data 수: {df_test.shape[0]}')
    print(f'submission data 수: {df_sub.shape[0]}')
    return df_train, df_test, df_sub

def df_ivf_di_split(df, is_train=False):

    data_type = 'train' if is_train else 'test'

    print(f'data 수: {df.shape[0]}')

    df_ivf = df[df['시술 유형'] == 'IVF'].drop(columns=['시술 유형'])
    df_di = df[df['시술 유형'] == 'DI'].drop(columns=['시술 유형'])

    print(f'시술유형 IVF {data_type} data 수: {df_ivf.shape[0]}')
    print(f'시술유형 DI {data_type} data 수: {df_di.shape[0]}')

    return df_ivf, df_di


def get_clf_eval(y_test, y_proba=None, fold_no=None):
    # Calculate and print evaluation metrics and confusion matrix,
    # accuracy, precision, recall, f1 and roc_auc score.
    # Optionally includes fold number in the output.

    # 임계값 0.5 기준 예측값 생성
    y_pred = (y_proba >= 0.5).astype(int)

    y_test = y_test.values

    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)  # ROC-AUC는 확률값 그대로 사용

    fold_info = f'Fold #{fold_no}' if fold_no is not None else ''
    print(f'{fold_info} ACC: {accuracy:.4f}, PRE: {precision:.4f}, REC: {recall:.4f}, F1: {f1:.4f}, ROC-AUC: {roc_auc:.4f}')
    return roc_auc


# Categorical variable encoding method.
def category_encoding(df_train, df_test, base_num_features, base_cat_features, config, is_ivf=False):
    target = '임신 성공 여부'

    data_name = 'IVF' if is_ivf else 'DI'

    new_df_train = pd.concat([df_train[base_num_features].copy(), df_train[target]], axis = 1)
    new_df_test = df_test[base_num_features].copy()

    print(f'{data_name} category 변수 인코딩: {config["encoding"]}')

    if config['encoding'] == 'target':
        encoder = ce.TargetEncoder(cols=base_cat_features)

    elif config['encoding'] == 'ordinal':
        encoder = ce.OrdinalEncoder(cols=base_cat_features)

    elif config['encoding'] == 'catboost':
        encoder = ce.CatBoostEncoder(cols=base_cat_features)#, random_state=config['seed'])

    if config['encoding'] in ['target', 'ordinal', 'catboost']:
        encoder.fit(df_train[base_cat_features], df_train[target])
        new_df_train[base_cat_features] = encoder.transform(df_train[base_cat_features])
        new_df_test[base_cat_features] = encoder.transform(df_test[base_cat_features])

    elif config['encoding'] == 'one-hot':
        encoder = ce.OneHotEncoder(cols=base_cat_features, use_cat_names = True)
        encoder.fit(df_train[base_cat_features], df_train[target])
        result_tr = encoder.transform(df_train[base_cat_features])
        result_te = encoder.transform(df_test[base_cat_features])

        result_tr.columns = result_tr.columns.str.replace(r'[^ㄱ-ㅎ가-힣A-Za-z0-9_]', '_', regex=True)
        result_te.columns = result_te.columns.str.replace(r'[^ㄱ-ㅎ가-힣A-Za-z0-9_]', '_', regex=True)

        new_df_train = pd.concat([new_df_train, result_tr], axis = 1)
        new_df_test = pd.concat([new_df_test, result_te], axis = 1)

    elif config['encoding'] is None:
        new_df_train = pd.concat([new_df_train, df_train[base_cat_features]], axis = 1)
        new_df_test = pd.concat([new_df_test, df_test[base_cat_features]], axis = 1)

    _, base_cat_features, base_features = make_feature_lists(new_df_train, is_ivf=is_ivf)

    return new_df_train, new_df_test, base_cat_features, base_features

def feature_engineering(df_input, tr_removed_column_list, is_train=False, is_ivf=False):
    df = df_input.copy()

    data_name = 'IVF' if is_ivf else 'DI'
    data_type = 'train' if is_train else 'test'

    if is_train:
        removed_column_list = []
    else:
        removed_column_list = tr_removed_column_list


    print(f'중복 제거 전 {data_type} {data_name} data 수: {df.shape[0]}')

    # drop_duplicates
    df = df.drop_duplicates(keep='first')

    print(f'중복 제거 후 {data_type} {data_name} data 수: {df.shape[0]}')

    # 횟수 관련 column
    count_columns = [column for column in df.columns if '횟수' in column]

    for col in count_columns:
        df[col] = df[col].str.replace('회', '').str.replace('이상', '').str.replace(' ', '').astype(int)


    # 시술 당시 나이 범위를 중앙값으로 변환하는 매핑 딕셔너리 (누락된 범위 추가)
    treatment_age = {
        "만18-34세": 26,
        "만35-37세": 36,
        "만38-39세": 38.5,
        "만40-42세": 41,
        "만43-44세": 43.5,
        "만45-50세": 47.5,
        "알 수 없음": -1  # 알 수 없음은 -1로 처리
    }

    # 공백 제거 후 매핑
    # 시술 당시 나이 숫자 mapping -> DI에만 적용
    if is_ivf:
        df["시술 당시 나이"] = df["시술 당시 나이"].str.strip().map(treatment_age).astype(float)

    # # 정자 난자기증자 나이 범위를 중앙값으로 변환하는 매핑 딕셔너리 (누락된 범위 추가)
    donor_age = {
        "만20세 이하": 20,
        "만21-25세": 23,
        "만26-30세": 28,
        "만31-35세": 33,
        "만36-40세": 38,
        "만41-45세": 43,
        "알 수 없음": -1 # 알 수 없음은 -1로 처리
    }

    # 공백 제거 후 매핑
    if is_ivf:
        df["난자 기증자 나이"] = df["난자 기증자 나이"].str.strip().map(donor_age).astype(float)

    df["정자 기증자 나이"] = df["정자 기증자 나이"].str.strip().map(donor_age).astype(float)

    # 특정 시술 유형
    treat_types = []
    # embryo_creation_reasons = []

    if not is_ivf:
        treat_types.extend(['IUI', 'ICI', 'Generic DI', 'IVI'])

    if is_ivf:
        treat_types.extend(['ICSI', 'IVF', 'BLASTOCYST', 'AH', 'Unknown'])
        # embryo_creation_reasons.extend(['현재 시술용', '배아 저장용', '기증용', '난자 저장용'])

    # 각 target에 대한 Multi-Hot Encoding 및 Count 계산
    for treat_type in treat_types:
        # 각 행에서 target이 등장하는 횟수 계산
        df[treat_type] = df['특정 시술 유형'].str.count(treat_type)

    # if is_ivf:
    #     for embryo_creation_reason in embryo_creation_reasons:
    #         # 각 행에서 target이 등장하는 횟수 계산
    #         df[embryo_creation_reason] = df['배아 생성 주요 이유'].str.count(embryo_creation_reason)

    df_column_list = df.columns
    print(f'column 제거 전 {data_type} {data_name} data shape: {df.shape}')

    if is_train:
        # train data에서 전부 결측치거나 값 1개만 갖는 column 제거

        for feat in df_column_list:

            null_count = df[feat].isna().sum() # feat 열 결측치 수

            if df[feat].nunique() == 0 or (df[feat].nunique() == 1 and null_count == 0):
                df = df.drop(columns=[feat])
                removed_column_list.append(feat)


    # # test data에서는 train data에서 제거된 column 제거
    else:

        df = df.drop(columns=removed_column_list)

    print(f'column 제거 후 {data_type} {data_name} data shape: {df.shape}')

    # 파생변수 생성
    # IVF에만 존재하는 피처
    if is_ivf:
        # 비율 및 비율 차이
        df["배아_생성률"] = df["미세주입에서 생성된 배아 수"] / (df["미세주입된 난자 수"] + 1e-5)
        df["배아_이식률"] = df["이식된 배아 수"] / (df["총 생성 배아 수"] + 1e-5)
        df["미세주입_이식률"] = df["미세주입 배아 이식 수"] / (df["미세주입에서 생성된 배아 수"] + 1e-5)
        df["배아_저장률"] = df["저장된 배아 수"] / (df["총 생성 배아 수"] + 1e-5)
        df["배아_해동률"] = df["해동된 배아 수"] / (df["저장된 배아 수"] + 1e-5)
        df["기증자_정자_비율"] = df["기증자 정자와 혼합된 난자 수"] / (df["혼합된 난자 수"] + 1e-5)
        df["배아_손실률"] = 1 - (df["미세주입에서 생성된 배아 수"] / (df["미세주입된 난자 수"] + 1e-5))
        df["배아_이식_대비_해동된_배아_비율"] = df["해동된 배아 수"] / (df["이식된 배아 수"] + 1e-5)
        df["미세주입_비율"] = df["미세주입된 난자 수"] / (df["총 생성 배아 수"] + 1e-5)
        df["신선_난자_저장률"] = df["저장된 신선 난자 수"] / (df["수집된 신선 난자 수"] + 1e-5)
        df["정자_혼합_비율_차이"] = (
            df["파트너 정자와 혼합된 난자 수"] / (df["혼합된 난자 수"] + 1e-5)
            - df["기증자 정자와 혼합된 난자 수"] / (df["혼합된 난자 수"] + 1e-5)
        )

        # 차이 및 변화량
        df["미세주입_실패_수"] = df["미세주입된 난자 수"] - df["미세주입에서 생성된 배아 수"]
        df["이식되지_않은_배아_수"] = df["미세주입에서 생성된 배아 수"] - df["미세주입 배아 이식 수"]
        df["저장되지_않은_신선_난자_수"] = df["수집된 신선 난자 수"] - df["저장된 신선 난자 수"]
        df["기증자_혼합_난자_수"] = df["혼합된 난자 수"] - df["파트너 정자와 혼합된 난자 수"]
        df["해동_후_이식_까지_시간"] = df["배아 이식 경과일"] - df["배아 해동 경과일"]
        df["해동_후_미세주입_성공률"] = df["미세주입된 난자 수"] / (df["해동된 배아 수"] + 1e-5)
        # df["배아_생성_감소량"] = df["미세주입된 난자 수"] - df["미세주입에서 생성된 배아 수"]
        # df["배아_해동_후_남은_기간"] = df["배아 이식 경과일"] - df["배아 해동 경과일"]
        df["배아_저장_후_해동_기간"] = df["배아 해동 경과일"] - df["난자 혼합 경과일"]
        df["정자_혼합_후_배아_생성_경과일"] = df["배아 이식 경과일"] - df["난자 혼합 경과일"]
        # df["미세주입_대비_해동된_배아_비율"] = df["미세주입된 난자 수"] / (df["해동된 배아 수"] + 1e-5)

    # 불임 원인 multi hot -> 하나의 feature

    # infertility_columns = [column for column in df.columns if '불임 원인' in column]

    # # dot()을 사용하여 1이 있는 컬럼을 결합 (각 feature 사이에 공백 추가)
    # df['불임 원인 종합'] = df[infertility_columns].dot(pd.Index([f'{col.replace("불임 원인", "").replace(" ", "")} ' for col in infertility_columns])).str.strip()

    # # 한글만 남기고 나머지 문자 제거 (정규 표현식 사용)
    # df['불임 원인 종합'] = df['불임 원인 종합'].apply(lambda x: re.sub(r'[^가-힣 ]', '', x))

    # # split()과 join()을 사용하여 공백을 ','로 변경
    # df['불임 원인 종합'] = df['불임 원인 종합'].apply(lambda x: ', '.join(x.split()))

    # df = df.drop(columns=infertility_columns)

    return df, removed_column_list

def make_feature_lists(df, is_ivf=False):
    base_features = []     # all features except target variable.
    base_num_features = [] # numerical features
    base_cat_features = [] # categorical features

    data_name = 'IVF' if is_ivf else 'DI'

    for feat in df.columns:
        # skip the target
        if feat == '임신 성공 여부':
            continue

        base_features.append(feat)

        if df[feat].dtype in ['object', 'category']:
            base_cat_features.append(feat)
        else:
            base_num_features.append(feat)

    # infertility_columns = [column for column in df.columns if '불임 원인' in column]

    # 공통 제거 변수
    removal_features = {
            'ID', '불임 원인 - 여성 요인', '불임 원인 - 정자 면역학적 요인', '불임 원인 - 자궁경부 문제',
            '시술 유형', '배란 유도 유형', '특정 시술 유형','배란 자극 여부','부부 주 불임 원인','여성 주 불임 원인'}

    # ivf 제거 변수
    removal_features_ivf = {
        '남성 주 불임 원인', '남성 부 불임 원인','부부 부 불임 원인','AH','신선_난자 저장률','IVF',
        '저장된 신선 난자 수', '신선 배아 사용 여부', '대리모 여부','난자 해동 경과일','DI 임신 횟수','불임 원인 - 정자 농도',
        '동결 배아 사용 여부','배아_저장_후_해동_기간','불임 원인 - 정자 운동성','DI 출산 횟수',
        '불임 원인 - 정자 형태', '기증 배아 사용 여부','배아 해동 경과일'
    }

    # di 제거 변수
    removal_features_di = {
        '불임 원인- 배란 장애','불명확 불임 원인',' 임신 시도 또는 마지막 임신 경과 연수','총 출산 횟수',
        '총 임신 횟수','시술 시기 코드'
    }
    if is_ivf:
        removal_features = removal_features.union(removal_features_ivf)
    else:
        removal_features = removal_features.union(removal_features_di)

    # removal_features.update(infertility_columns)

    # remove the specified features
    base_num_features = [i for i in base_num_features if i not in removal_features]
    base_cat_features = [i for i in base_cat_features if i not in removal_features]
    base_features = [i for i in base_features if i not in removal_features]

    print(f'{data_name} numeric feature 수: {len(base_num_features)}')
    print(f'{data_name} category feature 수: {len(base_cat_features)}')
    print(f'{data_name} 전체 feature 수: {len(base_features)}')

    return base_num_features, base_cat_features, base_features

def filling_missing_values(df_input, base_cat_features, base_num_features, config):
    df = df_input.copy()

    # Fill missing values for categorical features with 'Unknown' and ensure their data type is string.
    for base_cat_feat in base_cat_features:
        df[base_cat_feat] = df[base_cat_feat].astype(str)
        df[base_cat_feat] = df[base_cat_feat].fillna('알 수 없음')

        if config['encoding'] is None:
            df[base_cat_feat] = df[base_cat_feat].astype('category')


    # Fill missing values for numerical features with -1.
    for base_num_feat in base_num_features:
        df[base_num_feat] = df[base_num_feat].fillna(-1)

    return df


def model_kfold(df, config, params, base_features, base_cat_features, is_ivf=False):
    target = '임신 성공 여부'

    data_name = 'IVF' if is_ivf else 'DI'

    skf = StratifiedKFold(n_splits=config['k_fold'], shuffle=True, random_state=config['seed'])
    models = []       # trained models
    roc_auc_scores = []  # roc_auc_scores for validation sets

    model_params = params[config['model']]

    for k_fold, (train_idx, valid_idx) in enumerate(skf.split(df[base_features], df[target])):
        print(f'Fold #{k_fold + 1}')
        X_train, y_train = df[base_features].iloc[train_idx], df[target].iloc[train_idx]
        X_valid, y_valid = df[base_features].iloc[valid_idx], df[target].iloc[valid_idx]

        if config['model'] == 'logistic':
            model = LogisticRegression(**model_params)

        elif config['model'] == 'et':
            model = ExtraTreesClassifier(**model_params)

        elif config['model'] == 'rf':
            model = RandomForestClassifier(**model_params)

        if config['model'] in ['logistic', 'et', 'rf']:
            model.fit(X_train, y_train)

        elif config['model'] == 'lgb':
            model = LGBMClassifier(**model_params)

            model.fit(
                X_train, y_train,
                eval_set=[(X_valid, y_valid)],
                eval_metric='auc',
                categorical_feature=base_cat_features, # specify categorical features
            )

        elif config['model'] == 'xgb':
            model = XGBClassifier(**model_params)

            model.fit(
                X_train, y_train,
                eval_set=[(X_valid, y_valid)],
                verbose = 100
            )

        elif config['model'] == 'cbt':
            model = CatBoostClassifier(**model_params)

            model.fit(
                X_train, y_train,
                eval_set=[(X_valid, y_valid)],
                cat_features=base_cat_features, # specify categorical features
            )

        # save the trained model
        models.append(model)

        # evaluate the model
        # --- train-set
        print(f'[{data_name} Train] ', end='')
        y_prob = model.predict_proba(X_train)[:, 1]
        _ = get_clf_eval(y_train, y_prob, k_fold + 1)

        # --- valid-set
        print(f'[{data_name} Valid] ', end='')
        y_prob = model.predict_proba(X_valid)[:, 1]
        roc_auc_score = get_clf_eval(y_valid, y_prob, k_fold + 1)

        roc_auc_scores.append(roc_auc_score)

    avg_roc_auc = np.mean(roc_auc_scores)
    var_roc_auc = np.var(roc_auc_scores)

    print(f'Avg. roc-auc of validset {data_name}: {avg_roc_auc}')
    print(f'Var. roc-auc of validset {data_name}: {var_roc_auc}')

    return models, avg_roc_auc, var_roc_auc

def kfold_submission(df_test_ivf, df_test_di, df_sub, models_ivf, models_di, config):
    feat_importance_path = f"{config['root']}/FeatureImportance"
    submission_path = f"{config['root']}/Submission"
    json_path = f"{config['root']}/Json"

    if not os.path.exists(feat_importance_path):
        os.makedirs(feat_importance_path)

    if not os.path.exists(submission_path):
        os.makedirs(submission_path)

    if not os.path.exists(json_path):
        os.makedirs(json_path)

    # get current date and time
    now = datetime.now()

    # record the year, month, day, hour, and minute for naming files.
    year = now.year
    month = now.month
    day = now.day
    hour = now.hour
    minute = now.minute

    # file format
    submission_time = f"{year:04d}{month:02d}{day:02d}_{hour:02d}{minute:02d}"[2:]
    target = 'probability'

    # apply feature engineering
    base_num_features_ivf, base_cat_features_ivf, base_features_ivf = make_feature_lists(df_test_ivf, is_ivf=True)
    df_test_ivf = filling_missing_values(df_test_ivf, base_cat_features_ivf, base_num_features_ivf, config)
    X_test_ivf = df_test_ivf[base_features_ivf]

    base_num_features_di, base_cat_features_di, base_features_di = make_feature_lists(df_test_di, is_ivf=False)
    df_test_di = filling_missing_values(df_test_di, base_cat_features_di, base_num_features_di, config)
    X_test_di = df_test_di[base_features_di]

    # dataframe for feature importances
    base_features = list(set(base_features_ivf).union(set(base_features_di)))

    df_feature_importance_all_ivf = pd.DataFrame({'features': base_features_ivf})
    df_feature_importance_all_di = pd.DataFrame({'features': base_features_di})

    y_probs_ivf = 0
    y_probs_di = 0

    for i, model_ivf in enumerate(models_ivf):
        y_probs_ivf += model_ivf.predict_proba(X_test_ivf)[:, 1] / len(models_ivf)

        # save feature importance of current model
        if config['model'] in ['logistic']:
            df_feature_importance_all_ivf[f'model_{i}'] = model_ivf.coef_.squeeze()

        elif config['model'] in ['et', 'rf', 'lgb', 'xgb']:
            df_feature_importance_all_ivf[f'model_{i}'] = model_ivf.feature_importances_

        elif config['model'] == 'cbt':
            df_feature_importance_all_ivf[f'model_{i}'] = model_ivf.get_feature_importance()

    for i, model_di in enumerate(models_di):
        y_probs_di += model_di.predict_proba(X_test_di)[:, 1] / len(models_di)

        # save feature importance of current model
        if config['model'] in ['logistic']:
            df_feature_importance_all_di[f'model_{i}'] = model_di.coef_.squeeze()

        elif config['model'] in ['et', 'rf', 'lgb', 'xgb']:
            df_feature_importance_all_di[f'model_{i}'] = model_di.feature_importances_

        elif config['model'] == 'cbt':
            df_feature_importance_all_di[f'model_{i}'] = model_di.get_feature_importance()

    df_sub.loc[df_test_ivf.index, target] = y_probs_ivf
    df_sub.loc[df_test_di.index, target] = y_probs_di

    # save submission file as CSV
    df_sub.to_csv(f"{submission_path}/{submission_time}_{config['model']}_{config['encoding']}_submission.csv", index=False)

    # compute avarege, rank
    df_feature_importance_all_ivf['average'] = df_feature_importance_all_ivf.iloc[:, 1:].mean(axis=1).values
    df_feature_importance_all_ivf['rank'] = df_feature_importance_all_ivf['average'].rank(ascending=False)
    df_feature_importance_all_ivf = df_feature_importance_all_ivf.sort_values(by='rank')

    df_feature_importance_all_di['average'] = df_feature_importance_all_di.iloc[:, 1:].mean(axis=1).values
    df_feature_importance_all_di['rank'] = df_feature_importance_all_di['average'].rank(ascending=False)
    df_feature_importance_all_di = df_feature_importance_all_di.sort_values(by='rank')

    # save the feature importance as CSV
    df_feature_importance_all_ivf.to_csv(f'{feat_importance_path}/feat_import_{submission_time}_{config["model"]}_IVF_{config["encoding"]}.csv', index=False)
    df_feature_importance_all_di.to_csv(f'{feat_importance_path}/feat_import_{submission_time}_{config["model"]}_DI_{config["encoding"]}.csv', index=False)

    # save parameters as JSON
    json_data = json.dumps(config, indent=4)

    with open(f'{json_path}/{submission_time}_{config["model"]}_{config["encoding"]}.json', 'w') as file:
        file.write(json_data)

### Optuna Objective

In [13]:
def objective(trial: optuna.Trial, config, df, base_features, base_cat_features):

    # 'rf': {
        # 'n_jobs': 1,
        # 'random_state': config['seed'],

    #     "n_estimators": trial.suggest_int("n_estimators", 100, 5000),
    #     "max_depth": trial.suggest_int("max_depth", 2, 30),
    #     "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
    #     "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
    #     "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None])
    # },

    if config['model'] == 'xgb':
        model_params = {
        'random_state': config['seed'],
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'n_jobs': 1,
        'early_stopping_rounds': 10,
        'enable_categorical': True,
        'tree_method': 'hist',

        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=1, log=True),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 1, 12, step=1, log=False),
        "min_child_weight": trial.suggest_float("min_child_weight", 0.1, 10, log=False),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0, log=False),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0, log=False),
        "gamma": trial.suggest_float("gamma", 1e-8, 10.0, log=True),
        "lambda": trial.suggest_float("lambda", 1e-8, 10.0, log=True),
        # "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True)
        }

    elif config['model'] == 'lgb':
        model_params =  {
        'random_state': config['seed'],
        'objective': 'binary',
        'n_jobs': 1,
        'verbosity': -1,
        'early_stopping_rounds': 10,
        'deterministic': True,

        "n_estimators": trial.suggest_int("n_estimators", 100, 2000, step=1, log=True),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 1024, step=1, log=True),
        "max_depth": trial.suggest_int("max_depth", 6, 14, step=1, log=False),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0, log=False),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0, log=False),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True)
    }

    elif config['model'] == 'cbt':
        model_params = {
        'random_seed': config['seed'],
        'objective': 'Logloss',
        'eval_metric': 'AUC',
        'auto_class_weights': 'Balanced',
        'verbose': 100,
        'early_stopping_rounds': 10,
        'task_type': 'CPU',
        'allow_writing_files': False,

        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=1, log=True),
        'max_depth': trial.suggest_int('max_depth', 1, 12, step=1, log=False),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-8, 10.0, log=True),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0, log=False),
        "random_strength": trial.suggest_float("random_strength", 0.1, 10.0, log=False),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0, log=False),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0, log=False)
    }

    target = '임신 성공 여부'

    data_name = 'IVF' if config['is_ivf'] else 'DI'

    skf = StratifiedKFold(n_splits=config['k_fold'], shuffle=True, random_state=config['seed'])
    models = []       # trained models
    roc_auc_scores = []  # roc_auc_scores for validation sets

    for k_fold, (train_idx, valid_idx) in enumerate(skf.split(df[base_features], df[target])):
        print(f'Fold #{k_fold + 1}')
        X_train, y_train = df[base_features].iloc[train_idx], df[target].iloc[train_idx]
        X_valid, y_valid = df[base_features].iloc[valid_idx], df[target].iloc[valid_idx]

        if config['model'] == 'logistic':
            model = LogisticRegression(**model_params)

        elif config['model'] == 'et':
            model = ExtraTreesClassifier(**model_params)

        elif config['model'] == 'rf':
            model = RandomForestClassifier(**model_params)

        if config['model'] in ['logistic', 'et', 'rf']:
            model.fit(X_train, y_train)

        elif config['model'] == 'lgb':
            model = LGBMClassifier(**model_params)

            model.fit(
                X_train, y_train,
                eval_set=[(X_valid, y_valid)],
                eval_metric='auc',
                categorical_feature=base_cat_features, # specify categorical features
            )

        elif config['model'] == 'xgb':
            model = XGBClassifier(**model_params)

            model.fit(
                X_train, y_train,
                eval_set=[(X_valid, y_valid)],
                verbose = 100
            )

        elif config['model'] == 'cbt':
            model = CatBoostClassifier(**model_params)

            model.fit(
                X_train, y_train,
                eval_set=[(X_valid, y_valid)],
                cat_features=base_cat_features, # specify categorical features
            )

        # evaluate the model
        # --- train-set
        print(f'[{data_name} Train] ', end='')
        y_prob = model.predict_proba(X_train)[:, 1]
        _ = get_clf_eval(y_train, y_prob, k_fold + 1)

        # --- valid-set
        print(f'[{data_name} Valid] ', end='')
        y_prob = model.predict_proba(X_valid)[:, 1]
        roc_auc_score = get_clf_eval(y_valid, y_prob, k_fold + 1)

        roc_auc_scores.append(roc_auc_score)

    avg_roc_auc = np.mean(roc_auc_scores)
    var_roc_auc = np.var(roc_auc_scores)

    print(f'Avg. roc-auc of validset {data_name}: {avg_roc_auc}')
    print(f'Var. roc-auc of validset {data_name}: {var_roc_auc}')

    return avg_roc_auc

### Main

In [14]:
def kfold_tuning(config, n_trials=100):

    config['seed'] = config['seed_list'][0]

    # set seed
    set_seed(config['seed'])

    # read data set
    df_train, df_test, df_sub = read_data(config)

    # ivf di split
    if config['is_ivf']:
        df_train, _ = df_ivf_di_split(df_train, is_train = True)
        df_test, _ = df_ivf_di_split(df_test, is_train = False)

    else:
        _, df_train = df_ivf_di_split(df_train, is_train = True)
        _, df_test = df_ivf_di_split(df_test, is_train = False)
    # feature engineering (train,test)

    df_train, removed_column_list = feature_engineering(df_train, None, is_train=True, is_ivf=config['is_ivf'])
    df_test, _ = feature_engineering(df_test, removed_column_list, is_train=False, is_ivf=config['is_ivf'])

    # feature list 생성 (train)
    base_num_features, base_cat_features, base_features = make_feature_lists(df_train, is_ivf=config['is_ivf'])

    # 결측치 처리
    df_train = filling_missing_values(df_train, base_cat_features, base_num_features, config)

    # encoding
    df_train, df_test, base_cat_features, base_features = category_encoding(df_train, df_test, base_num_features, base_cat_features, config, is_ivf=config['is_ivf'])

    # check model performance
    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
    study.optimize(
        functools.partial(
            objective,
            config=config,
            df=df_train,
            base_features=base_features,
            base_cat_features=base_cat_features
        ),
        n_trials=n_trials  # Adjust the number of trials as needed
    )


    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    best_trial = study.best_trial

    print("  Value: ", best_trial.value)
    print("  Params: ")
    for key, value in best_trial.params.items():
        print(f"    {key}: {value}")

In [15]:
kfold_tuning(config, n_trials=100)

train data 수: 256351
test data 수: 90067
submission data 수: 90067
data 수: 256351
시술유형 IVF train data 수: 250060
시술유형 DI train data 수: 6291
data 수: 90067
시술유형 IVF test data 수: 87891
시술유형 DI test data 수: 2176
중복 제거 전 train IVF data 수: 250060
중복 제거 후 train IVF data 수: 250060
column 제거 전 train IVF data shape: (250060, 72)
column 제거 후 train IVF data shape: (250060, 71)
중복 제거 전 test IVF data 수: 87891
중복 제거 후 test IVF data 수: 87891
column 제거 전 test IVF data shape: (87891, 71)
column 제거 후 test IVF data shape: (87891, 70)
IVF numeric feature 수: 60
IVF category feature 수: 4
IVF 전체 feature 수: 64


[I 2025-02-26 13:09:43,617] A new study created in memory with name: no-name-8de1acba-b7bc-4541-b5a5-e9768ec2df49


IVF category 변수 인코딩: None
IVF numeric feature 수: 60
IVF category feature 수: 4
IVF 전체 feature 수: 64
Fold #1
[IVF Train] Fold #1 ACC: 0.7474, PRE: 0.5967, REC: 0.1067, F1: 0.1811, ROC-AUC: 0.7496
[IVF Valid] Fold #1 ACC: 0.7439, PRE: 0.5599, REC: 0.0975, F1: 0.1660, ROC-AUC: 0.7363
Fold #2
[IVF Train] Fold #2 ACC: 0.7384, PRE: 0.0000, REC: 0.0000, F1: 0.0000, ROC-AUC: 0.7335
[IVF Valid] Fold #2 ACC: 0.7384, PRE: 0.0000, REC: 0.0000, F1: 0.0000, ROC-AUC: 0.7327
Fold #3
[IVF Train] Fold #3 ACC: 0.7384, PRE: 0.0000, REC: 0.0000, F1: 0.0000, ROC-AUC: 0.7337
[IVF Valid] Fold #3 ACC: 0.7384, PRE: 0.0000, REC: 0.0000, F1: 0.0000, ROC-AUC: 0.7313
Fold #4
[IVF Train] Fold #4 ACC: 0.7473, PRE: 0.6057, REC: 0.0974, F1: 0.1679, ROC-AUC: 0.7477
[IVF Valid] Fold #4 ACC: 0.7417, PRE: 0.5398, REC: 0.0871, F1: 0.1500, ROC-AUC: 0.7375
Fold #5
[IVF Train] 

[I 2025-02-26 13:12:44,045] Trial 0 finished with value: 0.7336611332658076 and parameters: {'n_estimators': 1586, 'learning_rate': 0.00307290882031451, 'num_leaves': 307, 'max_depth': 7, 'min_child_samples': 27, 'subsample': 0.7126567953152935, 'colsample_bytree': 0.8985598942134658, 'reg_alpha': 0.0006180078549208188, 'reg_lambda': 1.7987095593907534}. Best is trial 0 with value: 0.7336611332658076.


Fold #5 ACC: 0.7384, PRE: 0.0000, REC: 0.0000, F1: 0.0000, ROC-AUC: 0.7346
[IVF Valid] Fold #5 ACC: 0.7384, PRE: 0.0000, REC: 0.0000, F1: 0.0000, ROC-AUC: 0.7304
Avg. roc-auc of validset IVF: 0.7336611332658076
Var. roc-auc of validset IVF: 7.75972480742976e-06
Fold #1
[IVF Train] Fold #1 ACC: 0.7384, PRE: 0.0000, REC: 0.0000, F1: 0.0000, ROC-AUC: 0.7388
[IVF Valid] Fold #1 ACC: 0.7384, PRE: 0.0000, REC: 0.0000, F1: 0.0000, ROC-AUC: 0.7300
Fold #2
[IVF Train] Fold #2 ACC: 0.7384, PRE: 0.0000, REC: 0.0000, F1: 0.0000, ROC-AUC: 0.7381
[IVF Valid] Fold #2 ACC: 0.7384, PRE: 0.0000, REC: 0.0000, F1: 0.0000, ROC-AUC: 0.7337
Fold #3
[IVF Train] Fold #3 ACC: 0.7384, PRE: 0.0000, REC: 0.0000, F1: 0.0000, ROC-AUC: 0.7383
[IVF Valid] Fold #3 ACC: 0.7384, PRE: 0.0000, REC: 0.0000, F1: 0.0000, ROC-AUC: 0.7314
Fold #4
[IVF Train] Fold #4 ACC: 0.7384, PRE: 0.0000, REC: 0.0000, F1: 0.0000, ROC-AUC: 0.7384
[IVF Valid] Fold #4 ACC: 0.7384, PRE: 0.0000, REC: 0.0000, F1: 0.0000, ROC-AUC: 0.7316
Fold #5
[I

...

Fold #5 ACC: 0.7430, PRE: 0.5424, REC: 0.1120, F1: 0.1857, ROC-AUC: 0.7361
Avg. roc-auc of validset IVF: 0.7377467356332827
Var. roc-auc of validset IVF: 1.6496430004988444e-06
Fold #1
[IVF Train] Fold #1 ACC: 0.7474, PRE: 0.5809, REC: 0.1229, F1: 0.2029, ROC-AUC: 0.7493
[IVF Valid] Fold #1 ACC: 0.7444, PRE: 0.5549, REC: 0.1167, F1: 0.1929, ROC-AUC: 0.7370
Fold #2
[IVF Train] Fold #2 ACC: 0.7485, PRE: 0.5908, REC: 0.1252, F1: 0.2066, ROC-AUC: 0.7488
[IVF Valid] Fold #2 ACC: 0.7426, PRE: 0.5387, REC: 0.1127, F1: 0.1864, ROC-AUC: 0.7400
Fold #3
[IVF Train] Fold #3 ACC: 0.7479, PRE: 0.5857, REC: 0.1238, F1: 0.2045, ROC-AUC: 0.7493
[IVF Valid] Fold #3 ACC: 0.7439, PRE: 0.5490, REC: 0.1178, F1: 0.1940, ROC-AUC: 0.7385
Fold #4
[IVF Train] Fold #4 ACC: 0.7477, PRE: 0.5869, REC: 0.1205, F1: 0.1999, ROC-AUC: 0.7479
[IVF Valid] Fold #4 ACC: 0.7421, PRE: 0.5349, REC: 0.1089, F1: 0.1810, ROC-AUC: 0.7378
Fold #5
[IVF Train] Fold #5 ACC: 0.7462, PRE: 0.5799, REC: 0.1077, F1: 0.1817, ROC-AUC: 0.7452


[I 2025-02-26 14:41:19,563] Trial 98 finished with value: 0.7378511581619638 and parameters: {'n_estimators': 471, 'learning_rate': 0.07229155431445283, 'num_leaves': 571, 'max_depth': 6, 'min_child_samples': 83, 'subsample': 0.6655096711959341, 'colsample_bytree': 0.5008869526299787, 'reg_alpha': 1.4008681313596558e-07, 'reg_lambda': 2.585679554058833e-07}. Best is trial 53 with value: 0.7381191446773872.


Fold #5 ACC: 0.7426, PRE: 0.5435, REC: 0.0999, F1: 0.1688, ROC-AUC: 0.7359
Avg. roc-auc of validset IVF: 0.7378511581619638
Var. roc-auc of validset IVF: 1.907381573532648e-06
Fold #1
[IVF Train] Fold #1 ACC: 0.7435, PRE: 0.5675, REC: 0.0815, F1: 0.1425, ROC-AUC: 0.7408
[IVF Valid] Fold #1 ACC: 0.7433, PRE: 0.5663, REC: 0.0803, F1: 0.1407, ROC-AUC: 0.7355
Fold #2
[IVF Train] Fold #2 ACC: 0.7384, PRE: 0.0000, REC: 0.0000, F1: 0.0000, ROC-AUC: 0.7278
[IVF Valid] Fold #2 ACC: 0.7384, PRE: 0.0000, REC: 0.0000, F1: 0.0000, ROC-AUC: 0.7290
Fold #3
[IVF Train] Fold #3 ACC: 0.7436, PRE: 0.5690, REC: 0.0823, F1: 0.1438, ROC-AUC: 0.7401
[IVF Valid] Fold #3 ACC: 0.7433, PRE: 0.5631, REC: 0.0829, F1: 0.1446, ROC-AUC: 0.7372
Fold #4
[IVF Train] Fold #4 ACC: 0.7443, PRE: 0.5757, REC: 0.0855, F1: 0.1490, ROC-AUC: 0.7406
[IVF Valid] Fold #4 ACC: 0.7416, PRE: 0.5415, REC: 0.0808, F1: 0.1406, ROC-AUC: 0.7369
Fold #5
[IVF Train] Fold #5 ACC: 0.7440, PRE: 0.5721, REC: 0.0848, F1: 0.1477, ROC-AUC: 0.7410
[

[I 2025-02-26 14:42:29,040] Trial 99 finished with value: 0.7348175245347509 and parameters: {'n_estimators': 274, 'learning_rate': 0.019809207212322925, 'num_leaves': 17, 'max_depth': 6, 'min_child_samples': 20, 'subsample': 0.7403873900268859, 'colsample_bytree': 0.562630934804206, 'reg_alpha': 1.4640475969953615e-06, 'reg_lambda': 3.669532031620491e-08}. Best is trial 53 with value: 0.7381191446773872.


Fold #5 ACC: 0.7427, PRE: 0.5556, REC: 0.0822, F1: 0.1432, ROC-AUC: 0.7355
Avg. roc-auc of validset IVF: 0.7348175245347509
Var. roc-auc of validset IVF: 8.979767679805894e-06
Number of finished trials:  100
Best trial:
  Value:  0.7381191446773872
  Params: 
    n_estimators: 572
    learning_rate: 0.03425368879706962
    num_leaves: 28
    max_depth: 7
    min_child_samples: 100
    subsample: 0.6969478337039123
    colsample_bytree: 0.5181370138590011
    reg_alpha: 0.007225084796211137
    reg_lambda: 2.9142363366332637e-07


In [ ]:
# read data set
df_train, df_test, df_sub = read_data(config)

# ivf di split
df_train_ivf, df_train_di = df_ivf_di_split(df_train, is_train = True)
df_test_ivf, df_test_di = df_ivf_di_split(df_test, is_train = False)

train data 수: 256351
test data 수: 90067
submission data 수: 90067
data 수: 256351
시술유형 IVF train data 수: 250060
시술유형 DI train data 수: 6291
data 수: 90067
시술유형 IVF test data 수: 87891
시술유형 DI test data 수: 2176


In [ ]:
unique_counts = {column: df_train_ivf[column].nunique() for column in df_train_ivf.columns}
for column, count in unique_counts.items():
    print(f"{column}: {count}")

In [ ]:
unique_counts = {column: df_train_di[column].nunique() for column in df_train_di.columns}
for column, count in unique_counts.items():
    print(f"{column}: {count}")

시술 시기 코드: 7
시술 당시 나이: 6
임신 시도 또는 마지막 임신 경과 연수: 19
특정 시술 유형: 5
배란 자극 여부: 2
배란 유도 유형: 1
단일 배아 이식 여부: 0
착상 전 유전 검사 사용 여부: 0
착상 전 유전 진단 사용 여부: 0
남성 주 불임 원인: 2
남성 부 불임 원인: 2
여성 주 불임 원인: 2
여성 부 불임 원인: 2
부부 주 불임 원인: 2
부부 부 불임 원인: 2
불명확 불임 원인: 2
불임 원인 - 난관 질환: 2
불임 원인 - 남성 요인: 2
불임 원인 - 배란 장애: 2
불임 원인 - 여성 요인: 1
불임 원인 - 자궁경부 문제: 1
불임 원인 - 자궁내막증: 2
불임 원인 - 정자 농도: 2
불임 원인 - 정자 면역학적 요인: 1
불임 원인 - 정자 운동성: 1
불임 원인 - 정자 형태: 2
배아 생성 주요 이유: 0
총 시술 횟수: 7
클리닉 내 총 시술 횟수: 7
IVF 시술 횟수: 7
DI 시술 횟수: 7
총 임신 횟수: 6
IVF 임신 횟수: 4
DI 임신 횟수: 6
총 출산 횟수: 4
IVF 출산 횟수: 3
DI 출산 횟수: 4
총 생성 배아 수: 0
미세주입된 난자 수: 0
미세주입에서 생성된 배아 수: 0
이식된 배아 수: 0
미세주입 배아 이식 수: 0
저장된 배아 수: 0
미세주입 후 저장된 배아 수: 0
해동된 배아 수: 0
해동 난자 수: 0
수집된 신선 난자 수: 0
저장된 신선 난자 수: 0
혼합된 난자 수: 0
파트너 정자와 혼합된 난자 수: 0
기증자 정자와 혼합된 난자 수: 0
난자 출처: 1
정자 출처: 1
난자 기증자 나이: 1
정자 기증자 나이: 7
동결 배아 사용 여부: 0
신선 배아 사용 여부: 0
기증 배아 사용 여부: 0
대리모 여부: 0
PGD 시술 여부: 0
PGS 시술 여부: 0
난자 채취 경과일: 0
난자 해동 경과일: 0
난자 혼합 경과일: 0
배아 이식 경과일: 0
배아 해동 경과일: 0
임신 성공 여부: 2
